In [ ]:
!python -c "import sys; print(sys.executable)"

In [1]:
import sys
print(sys.version)

3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]


In [2]:
!pip install mlflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import mlflow
import pandas as pd
pd.__version__

'1.5.3'

In [4]:
%%bash --bg

mlflow server --host 0.0.0.0 \
    --port 4000 \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./mlruns

In [5]:
%%bash
source mlflow_env_vars.sh
mlflow run .

2023/05/08 15:17:58 INFO mlflow.utils.conda: Conda environment mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 already exists.
2023/05/08 15:17:58 INFO mlflow.projects.utils: === Created directory /tmp/tmp7_cpisrz for downloading remote URIs passed to arguments of type 'path' ===
2023/05/08 15:17:58 INFO mlflow.projects.backend.local: === Running command 'source /home/huzefaanver/anaconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 1>&2 && python train.py 5' in run with ID 'b4ca631152e3407698b306eb27369436' === 
/home/huzefaanver/anaconda3/envs/mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'sklearn_knn' already exists. Creating a new version of this model...
2023/05/08 15:18:01 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for mod

In [6]:
%%bash
last_model_path=$(ls -tr mlruns/0/ | tail -1)
cat mlruns/0/$last_model_path/artifacts/knn/MLmodel

artifact_path: knn
flavors:
  python_function:
    env:
      conda: conda.yaml
      virtualenv: python_env.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.11.3
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.2.2
mlflow_version: 2.3.1
model_uuid: 6904d6ebea9a4814a6df3deb1d7d1a3b
run_id: 9aeee56b05d04b2ea5f7189cb2a236b3
utc_time_created: '2023-05-08 10:18:05.447946'


In [7]:
import mlflow
mlflow.__version__

'2.3.1'

In [8]:
%%bash --bg
source mlflow_env_vars.sh
mlflow --version
mlflow models serve -m models:/sklearn_knn/Production -p 8015 --env-manager=conda

In [9]:
import pandas as pd
from sklearn.datasets import load_wine

data = load_wine()
x = pd.DataFrame(data.data, columns=data.feature_names)
target = pd.DataFrame(data.target)
x['target'] = target

dataset = x
x


alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0      14.23        1.71  2.43               15.6      127.0           2.80   
1      13.20        1.78  2.14               11.2      100.0           2.65   
2      13.16        2.36  2.67               18.6      101.0           2.80   
3      14.37        1.95  2.50               16.8      113.0           3.85   
4      13.24        2.59  2.87               21.0      118.0           2.80   
..       ...         ...   ...                ...        ...            ...   
173    13.71        5.65  2.45               20.5       95.0           1.68   
174    13.40        3.91  2.48               23.0      102.0           1.80   
175    13.27        4.28  2.26               20.0      120.0           1.59   
176    13.17        2.59  2.37               20.0      120.0           1.65   
177    14.13        4.10  2.74               24.5       96.0           2.05   

     flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0          3.06                  0.28             2.29             5.64  1.04   
1          2.76                  0.26             1.28             4.38  1.05   
2          3.24                  0.30             2.81             5.68  1.03   
3          3.49                  0.24             2.18             7.80  0.86   
4          2.69                  0.39             1.82             4.32  1.04   
..          ...                   ...              ...              ...   ...   
173        0.61                  0.52             1.06             7.70  0.64   
174        0.75                  0.43             1.41             7.30  0.70   
175        0.69                  0.43             1.35            10.20  0.59   
176        0.68                  0.53             1.46             9.30  0.60   
177        0.76                  0.56             1.35             9.20  0.61   

     od280/od315_of_diluted_wines  proline  target  
0                            3.92   1065.0       0  
1                            3.40   1050.0       0  
2                            3.17   1185.0       0  
3                            3.45   1480.0       0  
4                            2.93    735.0       0  
..                            ...      ...     ...  
173                          1.74    740.0       2  
174                          1.56    750.0       2  
175                          1.56    835.0       2  
176                          1.62    840.0       2  
177                          1.60    560.0       2  

[178 rows x 14 columns]

In [10]:
x.iloc[-1].tolist()[:-1]

[14.13, 4.1, 2.74, 24.5, 96.0, 2.05, 0.76, 0.56, 1.35, 9.2, 0.61, 1.6, 560.0]

# Prediction on data that has original target == 2 

In [11]:

%%bash
data='[[14.13, 4.1, 2.74, 24.5, 96.0, 2.05, 0.76, 0.56, 1.35, 9.2, 0.61, 1.6, 560.0]]'
echo $data

curl -d "{\"inputs\": $data}" -H 'Content-Type: application/json' 127.0.0.1:8015/invocations

[[14.13, 4.1, 2.74, 24.5, 96.0, 2.05, 0.76, 0.56, 1.35, 9.2, 0.61, 1.6, 560.0]]
{"predictions": [2]}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100    20  100    91   8695  39565 --:--:-- --:--:-- --:--:-- 55500


We can see that the prediction results are correct.

# Prediction on data that has original target == 0

In [12]:
%%bash
data='[[14.23,1.71,2.43,5.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0],[13.2,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050.0]]'
echo $data

curl -d "{\"inputs\": $data}" -H 'Content-Type: application/json' 127.0.0.1:8015/invocations

[[14.23,1.71,2.43,5.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0],[13.2,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050.0]]
{"predictions": [0, 0]}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   174  100    23  100   151  14162  92980 --:--:-- --:--:-- --:--:--  169k


Prediction results are correct for this one as well.

## it is pertinent to mention that the input data passed for prediction was removed from the training dataset.

# META DATA AS LOGGED ON MLFLOW DASHBOARD
![](mlflow_metadata.png)